# Import Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy import stats
import seaborn as sns
sns.set_context('paper')
import os

# Set Paths

In [2]:
data_path='/Users/danielbabin/Documents/GitHub/T2_Timing/Data/'
figure_path='/Users/danielbabin/Documents/GitHub/T2_Timing/Figures/'

# Import Data
## Dome Fuji
Timescale + Gases and Temperature

## EDC Gas
We know the gas-temperature-snow phasing is good in this core

In [26]:
d15n=pd.read_csv(data_path+'edc2013d15n.txt',header=131,sep='\t',index_col='depth_m')
co2=pd.read_csv(data_path+'edc2013co2.txt',header=131,sep='\t',index_col='depth_m')
air_d18o=pd.read_csv(data_path+'edc2013d18oatm.txt',header=131,sep='\t',index_col='depth_m')

## Chinese Speleothem
Low Latitude Hydrologic Cycle, sensitive to abrupt climate change

## Coral Terrace Dates
Sea Level

## Orbital Solutions